In [1]:
import sqlite3

# c = conn.cursor()
db = sqlite3.connect('realtime.db',check_same_thread=False)
cursor = db.cursor()

# Retrieve list of all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Drop each table
for table in tables:
    print(f"Dropping table {table[0]}")
    cursor.execute(f'DROP TABLE {table[0]};')

# Commit changes and close connection
db.commit()
db.close()

print("All tables have been dropped.")

Dropping table BANKNIFTY24JUNFUT
Dropping table BANKNIFTY24JUNFUTcandles
Dropping table BANKNIFTY24JUN53000CE
Dropping table BANKNIFTY24JUN53000CEcandles
Dropping table BANKNIFTY24JUN53000PE
Dropping table BANKNIFTY24JUN53000PEcandles
Dropping table BANKNIFTY24JUN52900CE
Dropping table BANKNIFTY24JUN52900CEcandles
Dropping table BANKNIFTY24JUN52900PE
Dropping table BANKNIFTY24JUN52900PEcandles
Dropping table BANKNIFTY24JUN52800CE
Dropping table BANKNIFTY24JUN52800CEcandles
Dropping table BANKNIFTY24JUN52800PE
Dropping table BANKNIFTY24JUN52800PEcandles
Dropping table BANKNIFTY24JUN52700CE
Dropping table BANKNIFTY24JUN52700CEcandles
Dropping table BANKNIFTY24JUN52700PE
Dropping table BANKNIFTY24JUN52700PEcandles
Dropping table BANKNIFTY24JUN52600CE
Dropping table BANKNIFTY24JUN52600CEcandles
Dropping table BANKNIFTY24JUN52600PE
Dropping table BANKNIFTY24JUN52600PEcandles
Dropping table BANKNIFTY24JUN52500CE
Dropping table BANKNIFTY24JUN52500CEcandles
Dropping table BANKNIFTY24JUN52500PE

In [10]:
import sqlite3
from datetime import datetime
# import pytz

# Establish a connection to the database
conn = sqlite3.connect('candledata.db')
cursor = conn.cursor()

# Get all table names from the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
# print(tables)

# Loop through all tables and add a new column for the Indian time
for table in tables:
    table_name = table[0]
    # Add new column for Indian formatted time
    cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN indian_time TEXT")

    # Update the new column with the time formatted as hh:mm:ss
    cursor.execute(f"""
        UPDATE {table_name}
        SET indian_time = (
            SELECT strftime('%H:%M:%S', datetime(Timestamp, 'unixepoch', 'localtime', '-3 hours', '30 minutes'))
            FROM {table_name} AS t2
            WHERE t2.rowid = {table_name}.rowid
        )
    """)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [9]:
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('candledata.db')
cursor = conn.cursor()

# Get all table names from the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    # Create a new table name
    new_table_name = f"{table_name}_new"

    # Fetch the old table's schema
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()

    # Formulate the new table's column definitions, excluding 'indian_time'
    new_columns = ', '.join(f"{col[1]} {col[2]}" for col in columns if col[1] != 'indian_time')
    
    # Create a new table with the same structure minus the 'indian_time' column
    cursor.execute(f"CREATE TABLE {new_table_name} ({new_columns})")
    
    # Prepare column names for data transfer, excluding 'indian_time'
    column_names = ', '.join(col[1] for col in columns if col[1] != 'indian_time')
    
    # Copy data from the old table to the new table
    cursor.execute(f"INSERT INTO {new_table_name} ({column_names}) SELECT {column_names} FROM {table_name}")
    
    # Drop the old table
    cursor.execute(f"DROP TABLE {table_name}")
    
    # Rename the new table to the original table name
    cursor.execute(f"ALTER TABLE {new_table_name} RENAME TO {table_name}")

# Commit the changes and close the connection
conn.commit()
conn.close()
